# Fine-Tuning von BERT für Named Entity Recognition (NER)

von Andreas Sünder & Benjamin Kissinger (2024)

Ziel dieses Tutorials ist es, ein BERT-Modell für Named Entity Recognition (NER) zu trainieren. Das fertige Modell soll für das Extrahieren von Metadaten aus Büchern, Literatur etc. verwendet werden können. Am Ende ist es in der Lage, aus einem Ausgangstext den Autor sowie das Veröffentlichungsjahr zu extrahieren.

## Setup

Zunächst sind die notwendigen Packages zu installieren:

```bash
pip install -r requirements.txt
```

## Laden des Datasets

Für dieses Tutorial nutzen wir diverse Libraries, die von *🤗 HuggingFace* via Python bereitgestellt werden, darunter finden sich *🤗 Datasets* und *🤗 Transformers*. Ersteres ist zum Interagieren mit Datensätzen, die auf der Platform *HuggingFace Hub* (https://huggingface.co/) frei zur Verfügung gestellt werden. Die Transformers-Library kann zum Interagieren (sprich Trainieren etc.) mit diversen Modellen verwendet werden.

Zunächst laden wir den passenden Datensatz, der unter https://huggingface.co/datasets/textminr/ner_tokenized verfügbar ist:

In [ ]:
from datasets import load_dataset

dataset = load_dataset('textminr/ner_tokenized')

## Vorbereitung des Datasets

Danach können wir den *Tokenizer* laden. Zu jedem Sprachmodell gibt es einen passenden Tokenizer, der die Eingabe in die passenden Tokens (das sind kleinere Einheiten bzw. Subwörter) aufsplittet. Hier müssen wir auch die Modellbezeichnung angeben, die wir verwenden wollen. In unserem Fall ist das `bert-base-multilingual-cased`:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
  'bert-base-multilingual-cased', # ID des Modells, um den richtigen Tokenizer zu laden
  add_prefix_space=True
)

Nun müssen wir unsere Trainingsdaten aus dem Dataset *tokenizen*, sprich jeden Satz in kleinere Einheiten aufteilen. Dafür verwenden wir eine eigene Funktion `tokenize_and_align_labels`:

In [ ]:
def tokenize_and_align_labels(row):
  tokenized_inputs = tokenizer(row['tokens'], truncation=True, is_split_into_words=True)

  labels = []
  for i, label in enumerate(row[f'ner_ids']):
    word_ids = tokenized_inputs.word_ids(batch_index=i)
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
      if word_idx is None:
        label_ids.append(-100)
      elif word_idx != previous_word_idx:
        label_ids.append(label[word_idx])
      else:
        label_ids.append(-100)
      previous_word_idx = word_idx
    labels.append(label_ids)

  tokenized_inputs['labels'] = labels
  return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

Beim Trainieren von Sprachmodellen ist es immer wichtig, dass **alle** Sätze die gleiche Länge haben. Dafür müssen wir die Sätze *padden*, sprich die kürzeren Sätze mit einem speziellen Token (am Anfang oder Ende) auffüllen. Dazu verwenden wir einen *DataCollator*, welchen wir später beim Trainieren mitgeben.

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

Um bewerten zu können, ob unser Modell "gut" ist, können wir während des Trainings die *Accuracy* berechnen. Nachdem in den Trainingsdaten die Labels für jedes Token enthalten sind, können wir die Accuracy berechnen, indem wir die Anzahl der richtigen Vorhersagen mit den wahren Labels vergleichen. Dafür verwenden wir eine eigene Funktion `compute_metrics`:

In [ ]:
import evaluate

seqeval = evaluate.load('seqeval')
import numpy as np

# Labels, die wir extrahieren wollen
label_list = ['O', 'AUTHOR', 'DATE']

def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax(predictions, axis=2)

  true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
  ]
  true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
  ]

  results = seqeval.compute(predictions=true_predictions, references=true_labels)
  return {
    'precision': results['overall_precision'],
    'recall': results['overall_recall'],
    'f1': results['overall_f1'],
    'accuracy': results['overall_accuracy'],
  }

## Train model

Nun haben wir alle Vorkehrungen getroffen, um das Modell zu trainieren! Zu Beginn müssen wir das Modell aber herunterladen und in den Speicher (RAM oder VRAM, wenn eine GPU verfügbar ist) laden. Wir müssen hier auch die Labels der Klassen angeben, die wir extrahieren wollen (wobei jedes Label auch eine Art "ID", sprich eine Zahl, hat):

In [ ]:
from transformers import AutoModelForTokenClassification

id2tag= {
  0: 'O',
  1: 'AUTHOR',
  2: 'DATE',
}
tag2id = {v: k for k, v in id2tag.items()}

model = AutoModelForTokenClassification.from_pretrained(
  'bert-base-multilingual-cased', # wieder die ID des Modells
  num_labels=len(label_list), # Anzahl der Labels
  id2label=id2tag, # Zuordnung der IDs zu den Labels
  label2id=tag2id, # Zuordnung der Labels zu den IDs
)

HuggingFace Transformers bietet eine einfache Möglichkeit an, um unser Modell zu trainieren. Wir definieren hierfür ein Objekt der Klasse `TrainingArguments` und übergeben unsere gewünschten Trainingseinstellungen (auch genannt *Hyperparameter*). Unsere `traininga-args` übergeben wir dann einem `Trainer` mitsamt den Datensätzen, der Evaluierungs-Funktion und dem Data Collator von vorhin (und noch ein paar andere Sachen). Das Training starten wir mit `trainer.train()`:

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
  output_dir='./output',
  per_device_eval_batch_size=4,
  per_device_train_batch_size=4,
  learning_rate=2e-5,
  num_train_epochs=1,
  logging_strategy='steps',
  logging_steps=25,
  evaluation_strategy='epoch',
  eval_steps=1,
  save_strategy='epoch',
  save_steps=1,
)
  
trainer = Trainer(
  model=model,
  args=training_args,
  tokenizer=tokenizer,
  train_dataset=tokenized_datasets['train'],
  eval_dataset=tokenized_datasets['validation'],
  data_collator=data_collator,
  compute_metrics=compute_metrics,
)

trainer.train()

## Inference

Haben wir unser Modell fertig trainiert, können wir es auch gleich testen. Vorher hatten wir das `output-dir` auf `./output` gesetzt, weshalb wir das Modell nun auch von dort laden (hier muss nur noch der Checkpoint-Ordner angepasst werden):

In [ ]:
from transformers import pipeline

classifier = pipeline(
  'token-classification',
  model='./output/checkpoint-<XXXX>', # Speicherort des Modells
  aggregation_strategy='simple'
)

In [ ]:
sentence = "Alan Walker's vibrant self-portraits, painted in 1940, express her tumultuous inner world."
classifier(sentence)